In [ ]:
!unzip '/content/train.zip'
!unzip '/content/validate.zip'
!unzip '/content/trainall.zip'
!unzip '/content/test.zip'
# Lema

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Sat Feb 20 19:06:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    30W /  70W |    352MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import pandas as pd
from imageio import imread
import math
import numpy as np
import cv2
import keras
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.utils import to_categorical

In [ ]:
Image_Width=128
Image_Height=64
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [ ]:
# # cate = {
# #     1:'Cấm ngược chiều',
# #     2:'Cấm dừng và đỗ',
# #     3:'Cấm rẽ',
# #     4:'Giới hạn tốc độ',
# #     5:'Cấm còn lại',
# #     6:'Nguy hiểm',
# #     7:'Hiệu lệnh'
# # }
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

train_dir = "/content/train/"
filenames_train=os.listdir(train_dir)
images_train=[]
categories_train=[]
images_name_train=[]
for dir in filenames_train:
    category = os.path.join(train_dir,dir)
    for f_name in os.listdir(category):
      image = imread(os.path.join(category, f_name))
      image = cv2.resize(image, Image_Size)
      if dir == 2 or dir == 3 or dir == 5:
        image = increase_brightness(image, value=30)
      images_train.append(image)
      images_name_train.append(f_name)
      # categories.append(cate[int(dir)])
      categories_train.append(dir)
train_df=pd.DataFrame({
    'images_name':images_name_train,
    'category':categories_train
})

validate_dir = "/content/validate/"
filenames_validate=os.listdir(validate_dir)
images_validate=[]
categories_validate=[]
images_name_validate=[]
for dir in filenames_validate:
    category = os.path.join(validate_dir,dir)
    for f_name in os.listdir(category):
      image = imread(os.path.join(category, f_name))
      image = cv2.resize(image, Image_Size)
      if dir == 2 or dir == 3 or dir == 5:
        image = increase_brightness(image, value=30)
      images_validate.append(image)
      images_name_validate.append(f_name)
      # categories.append(cate[int(dir)])
      categories_validate.append(dir)
validate_df=pd.DataFrame({
    'images_name':images_name_validate,
    'category':categories_validate
})

In [ ]:
filter_size = (3,3)
pool_size = (2,2)
model = Sequential([
        Conv2D(64, filter_size, activation='relu', input_shape=(Image_Width,Image_Height,Image_Channels), padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=pool_size),
        Conv2D(16, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        Dropout(0.2),
        Conv2D(16, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=pool_size),
        Conv2D(32, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        Dropout(0.2),
        Conv2D(32, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(7, activation='softmax')
])
# model = Sequential()
# model.add(Conv2D(8, (3, 3), input_shape=(Image_Width,Image_Height,Image_Channels),activation="relu"))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(16, (3, 3), activation="relu"))
# model.add(BatchNormalization())
# model.add(Conv2D(16, (3, 3), activation="relu"))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
# model.add(BatchNormalization())
# model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
# model.add(BatchNormalization())
# model.add(Flatten())
# model.add(Dense(1024, activation="relu"))
# model.add(Dropout(0.3))
# model.add(Dense(512, activation="relu"))
# model.add(Dropout(0.2))
# model.add(Dense(7, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 128, 64, 8)        224       
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 64, 8)        32        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 32, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 32, 16)        1168      
_________________________________________________________________
batch_normalization_6 (Batch (None, 64, 32, 16)        64        
_________________________________________________________________
dropout_3 (Dropout)          (None, 64, 32, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 32, 16)       

In [ ]:
batch_size = 8
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "/content/trainall",x_col='images_name',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size, subset = "training")
validation_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.9999999999)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "/content/trainall", 
    x_col='images_name',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size, subset = "validation")

Found 2948 validated image filenames belonging to 7 classes.
Found 324 validated image filenames belonging to 7 classes.


In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [ ]:
# from keras.callbacks import LearningRateScheduler
epochs = 10
# lr_sched = LearningRateScheduler(lambda epoch: 5e-5 * (0.75 ** np.floor(epoch / 2)))
history = model.fit_generator(train_generator, epochs = epochs, validation_data=validation_generator)
#,validation_steps=total_validate//batch_size,
# steps_per_epoch=total_train//batch_size,
# callbacks=callbacks)

Epoch 1/10
  1/369 [..............................] - ETA: 53s - loss: 1.4499e-04 - accuracy: 1.0000

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


369/369 [==============================] - 4s 9ms/step - loss: 0.1276 - accuracy: 0.9841 - val_loss: 0.9216 - val_accuracy: 0.9537
Epoch 2/10
369/369 [==============================] - 3s 9ms/step - loss: 0.1297 - accuracy: 0.9841 - val_loss: 0.5712 - val_accuracy: 0.9568
Epoch 3/10
369/369 [==============================] - 3s 9ms/step - loss: 0.1607 - accuracy: 0.9800 - val_loss: 0.9796 - val_accuracy: 0.9259
Epoch 4/10
369/369 [==============================] - 3s 9ms/step - loss: 0.0819 - accuracy: 0.9881 - val_loss: 0.7046 - val_accuracy: 0.9506
Epoch 5/10
369/369 [==============================] - 3s 9ms/step - loss: 0.0874 - accuracy: 0.9919 - val_loss: 1.0700 - val_accuracy: 0.9506
Epoch 6/10
369/369 [==============================] - 3s 9ms/step - loss: 0.0999 - accuracy: 0.9871 - val_loss: 1.5441 - val_accuracy: 0.9599
Epoch 7/10
369/369 [==============================] - 3s 9ms/step - loss: 0.0690 - accuracy: 0.9936 - val_loss: 1.5460 - val_accuracy: 0.9691
Epoch 8/10
369/36

In [ ]:
model.load_weights('/content/drive/MyDrive/modeltraffic.h5')

In [ ]:
# tạo tập test và vứt predict ra file traffic.csv
test_datagen = ImageDataGenerator(rescale=1./255)

test_dir = '/content/test'

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size= Image_Size,
        class_mode='categorical',
        shuffle = False,
        batch_size= 1)
filenames = test_generator.filenames
print(filenames)
nb_samples = len(filenames)
predict = model.predict_generator(test_generator,steps = nb_samples)
omg = np.empty(0)
for i in range (nb_samples):
  maxx = predict[i][0]
  index = 0
  for j in range (1,7):
    if maxx < predict[i][j]:
      maxx = max(maxx,predict[i][j])
      index = j
  omg = np.append(omg,int(index)+1)   
pd.DataFrame(omg).to_csv("/content/drive/MyDrive/Traffic1.csv")

Found 2200 images belonging to 1 classes.
['test/10000.png', 'test/10011.png', 'test/10016.png', 'test/10024.png', 'test/10032.png', 'test/10036.png', 'test/10038.png', 'test/1005.png', 'test/10050.png', 'test/10053.png', 'test/10065.png', 'test/10066.png', 'test/10070.png', 'test/10078.png', 'test/10081.png', 'test/10088.png', 'test/10094.png', 'test/10107.png', 'test/10108.png', 'test/1011.png', 'test/10117.png', 'test/10121.png', 'test/10122.png', 'test/10124.png', 'test/10131.png', 'test/10137.png', 'test/10138.png', 'test/10139.png', 'test/10144.png', 'test/10151.png', 'test/10152.png', 'test/10156.png', 'test/10159.png', 'test/10163.png', 'test/10166.png', 'test/10175.png', 'test/10181.png', 'test/10184.png', 'test/10188.png', 'test/10199.png', 'test/10210.png', 'test/10215.png', 'test/10217.png', 'test/10220.png', 'test/10222.png', 'test/10223.png', 'test/10231.png', 'test/10237.png', 'test/10238.png', 'test/1024.png', 'test/10243.png', 'test/10263.png', 'test/10279.png', 'test/

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [ ]:
#lấy toàn bộ tên các ảnh trong tập test để đối xứng sang kq trên
dir = "/content/test/test"
filenames=os.listdir(dir)
filenames.sort()
print(filenames)
pd.DataFrame(filenames).to_csv("/content/drive/MyDrive/Title.csv")

['10000.png', '10011.png', '10016.png', '10024.png', '10032.png', '10036.png', '10038.png', '1005.png', '10050.png', '10053.png', '10065.png', '10066.png', '10070.png', '10078.png', '10081.png', '10088.png', '10094.png', '10107.png', '10108.png', '1011.png', '10117.png', '10121.png', '10122.png', '10124.png', '10131.png', '10137.png', '10138.png', '10139.png', '10144.png', '10151.png', '10152.png', '10156.png', '10159.png', '10163.png', '10166.png', '10175.png', '10181.png', '10184.png', '10188.png', '10199.png', '10210.png', '10215.png', '10217.png', '10220.png', '10222.png', '10223.png', '10231.png', '10237.png', '10238.png', '1024.png', '10243.png', '10263.png', '10279.png', '10281.png', '10286.png', '10293.png', '10294.png', '10300.png', '10303.png', '10306.png', '10310.png', '10327.png', '10329.png', '1034.png', '10342.png', '10349.png', '10351.png', '10352.png', '10355.png', '10365.png', '10371.png', '1038.png', '10384.png', '10394.png', '10395.png', '10396.png', '10402.png', '10